# Etape 1 : preprocessing DVF

### 1.1. Importation du dataset DVF :

Nous importons le dataset « **Demandes de valeurs foncières** » (DVF), publié par la DGFiP, permet de connaître les transactions immobilières intervenues au cours des cinq dernières années sur le territoire métropolitain et les DOM-TOM, à l’exception de l’Alsace, de la Moselle et de Mayotte. Les données contenues sont issues des actes notariés et des informations cadastrales.

Fichiers 2017-2020 : https://files.data.gouv.fr/geo-dvf/latest/

In [ ]:
name = "https://files.data.gouv.fr/geo-dvf/latest/csv/2021/full.csv.gz"

def load_data():
    data = pd.read_csv(name, sep = ',')
    """
    for year in range(2017, 2021):
        name = "https://files.data.gouv.fr/geo-dvf/latest/csv/" + str(year) + "/full.csv.gz"
        table = pd.concat([table, pd.read_csv(name, sep = ',')])

    display("Taille de table :")
    display(table.shape)
    table.head()
    """
    return data

table = load_data()

# On ne travaille que sur les données du S1 2021, afin d'avoir des calculs moins coûteux en temps.
# Les transactions du S1 2021 représentent tout de même un dataset de 1 200 000 lignes...
# Pour travailler sur l'ensemble des données (2017-2021), il suffit d'enlever les guillemets ci-dessus.

### 1.2. Visualisation des données DVF

Il s'agit dans cette section de se faire **des premières intuitions sur les données**. 

En particulier, on se rend compte d'un **problème de preprocessing** : une transaction peut correspondre à plusieurs lignes avec la même valeur foncière sur chaque ligne. Autrement dit, **DVF affiche le même prix de vente global à chaque lot d'une même transaction**.

Pour observer cela, nous allons créer **un identifiant de transaction unique**.

In [ ]:
def sales_id(table):

    # Création d'une adresse générique
    table['adresse_numero'] = table['adresse_numero'].fillna('0').astype(int)
    table['adresse_suffixe'] = table['adresse_suffixe'].fillna(' ')
    table['adresse_code_voie'] = table['adresse_code_voie'].fillna(' ')
    table['adresse_nom_voie'] = table['adresse_nom_voie'].fillna(' ')
    table['code_postal'] = table['code_postal'].fillna('0').astype(int)
    table['nom_commune'] = table['nom_commune'].fillna(' ')

    # Ajout de "\" pour que l'opération soit visible à l'écran en entier
    table["adresse"] = table['adresse_numero'].astype(str) + ' ' + table['adresse_suffixe'] + ' ' + \
                    table['adresse_code_voie'] + ' ' + table['adresse_nom_voie'] + ' ' + table['nom_commune'] + ' ' + \
                    table['code_postal'].astype(str) + ' ' + 'France'

    # Création d'un identifiant de transaction :
    # Pour identifier les doublons, l'adresse ne suffit pas : un bien peut avoir été vendu deux fois dans la même année
    table["identifiant_transaction"] = table["adresse"].astype(str) + ' le ' + table["date_mutation"].astype(str)
    
    return table

table = sales_id(table)

In [ ]:
# Problème dans les données et vérification de la validité de l'identifiant de transaction :

# display(table["identifiant_transaction"].loc[0])
# display(table["identifiant_transaction"].loc[1])
# display("Si l'identifiant de transaction est valide, alors True doit s'afficher :")
# table["identifiant_transaction"].loc[0] == table["identifiant_transaction"].loc[1]

In [ ]:
# On constate, encore une fois, le problème relevé ci-dessus :

# display("Nombre d'adresses uniques dans le DataFrame :")
# display(len(table["adresse"].unique()))

# display("Nombre d'identifiant_transaction uniques dans le DataFrame :")
# display(len(table["identifiant_transaction"].unique()))

# display("Nombre de lignes dans le DataFrame :")
# display(len(table))

# display("Nombre moyen de lignes par vente :")
# np.round(len(table) / len(table["identifiant_transaction"].unique()), 2)

# Une vente correspond à plusieurs lignes, les informations sont donc diffusées dans ces lignes...

Dès lors, si on entraîne l'algorithme de pricing sur ce dataset, il sera **biaisé** : 
* d'une part, il associerait à une dépendance de 20 m2 le prix d'un appartement de 200 m2
* d'autre part, il ne prendrait pas en compte la plus-value apportée par un jardin à une maison, par une dépendance à un appartement, etc.

Il conviendra donc de **retravailler les données pour obtenir une seule ligne par transaction**.

**Le notebook "cleaning-dvf"** (lien : https://github.com/victor-kerros/tokenisation-immo/blob/main/cleaning-dvf.ipynb) revient en détail sur ce problème de preprocessing et effectue ce travail de nettoyage. Dans la mesure où cette solution de nettoyage est très coûteuse en temps, nous privilégions **une solution plus efficace** dans le cadre de ce projet : nous ne retenons que les transactions ne faisant l'objet que d'une seule ligne.

### 1.3. Création du dataset final DVF

D'abord, nous ne prenons que les colonnes suivantes :
- Date de vente/mutation
- Nature mutation (pour séparer les ventes en VEFA et les ventes classiques)
- Valeur foncière (prix de vente)
- Colonnes liées à l’adresse (pour nous permettre de localiser le bien)
- Adresse
- Code Postal
- Type local (maison/appartement/Local commercial/Dépendance etc)
- Surface réelle bâtie (nb de mètre carré du bien bâti)
- Surface terrain (nb de mètre carré du terrain associé au bien)

Ensuite, comme annoncé, **nous ne retenons que les transactions ne faisant l'objet que d'une seule ligne pour créer "data"** : le dataset final avec lequel nous allons travailler.

In [1]:
def create_data_vf(table):

    # On crée le dataframe table_vf avec les seules colonnes qui nous intéressent
    colonnes = ["date_mutation", "nature_mutation", "valeur_fonciere", "code_postal", 'type_local',
                'surface_reelle_bati', 'nombre_pieces_principales', 'nature_culture', 'surface_terrain', 'longitude', 
                'latitude', 'adresse', 'code_departement', 'identifiant_transaction']
    table_vf = table[colonnes].copy()

    # On agrège les types de cultures différents de NaN, sols, terrain à bâtir et  : on les renomme "culture"
    culture_type = ['taillis simples', 'eaux', 'landes', 'taillis sous futaie', 'prés', 'terres', 'peupleraies', 
                    'vignes', 'bois', 'vergers', 'carrières', 'futaies résineuses', 'pâtures', 'futaies feuillues', 
                    'futaies mixtes', 'chemin de fer', 'oseraies', 'pacages', 'prés plantes', 'terres plantées', 
                    'landes boisées', 'herbages', "prés d'embouche"]

    for x in culture_type:
        table_vf.loc[table_vf["nature_culture"] == x, "nature_culture"] = "culture"

    # On ne retient que les transactions ayant fait l'objet d'une seule et unique ligne
    table_vf_dup = table_vf.copy()
    table_vf_uni = table_vf.copy()

    # On récupère les indices des transactions dupliquées...
    # i.e. les lignes où on retrouve un id de transaction utilisé ailleurs
    dup_id = table_vf_dup.groupby('identifiant_transaction').size()
    dup_id = dup_id[dup_id > 1]
    dup_id = dup_id.reset_index()

    table_vf_dup = table_vf_dup[table_vf_dup['identifiant_transaction'].isin(dup_id["identifiant_transaction"])]
    table_vf_uni = table_vf_uni[~table_vf_uni['identifiant_transaction'].isin(dup_id["identifiant_transaction"])]

    # Décommenter les lignes ci-dessous pour obtenir le nombre de transactions non dupliquées
    # print("Taille de table_vf_uni (nombre de transactions non dupliquées) :")
    # print(table_vf_uni.shape)

    # La solution la plus efficace, pour éviter un nettoyage trop coûteux en temps...
    # est de ne retenir que les transactions non dupliquées.
    data = table_vf_uni
    data = data.reset_index().drop("index", axis = 1)
    
    return data

data = create_data_vf(table)

# Visualisation de data (trois premières lignes) :
data.head(3)

NameError: name 'table' is not defined

### 1.4. Valeurs extrêmes dans le dataset DVF

On s'intéresse aux **valeurs extrêmes dans le dataset**. En effet, afin d'avoir un entraînement fiable, nous devons les enlever (sinon, il y aura trop de bruit).

In [ ]:
def boxplot_display(data):

    fig, axs = plt.subplots(1,4)
    fig.suptitle("Boxplot des variables d'intérêt :")

    axs[0].boxplot(data[data['valeur_fonciere'].notna()]['valeur_fonciere'])
    axs[0].set(title = "Valeurs foncières")

    axs[1].boxplot(data[data['surface_reelle_bati'].notna()]['surface_reelle_bati'])
    axs[1].set(title = "Surface bati")

    axs[2].boxplot(data[data['nombre_pieces_principales'].notna()]['nombre_pieces_principales'])
    axs[2].set(title = "Nbre pièces")

    axs[3].boxplot(data[data['surface_terrain'].notna()]['surface_terrain'])
    axs[3].set(title = "Surface terrain")

    fig.tight_layout()

    plt.show()

In [ ]:
# boxplot_display(data)

In [ ]:
# ax = sns.boxplot(x = "valeur_fonciere", y = "type_local", data = data)

On constate sur ces boxplots que le dataset présente des valeurs extrêmes **particulièrement hautes** dans les quatre variables d'intérêt. 

En particulier, les locaux industriels, commerciaux et assimilés ont des valeurs extrêmes très importantes.

On observe également des valeurs extrêmes **particulièrement basses**, comme le montrent les cellules ci-dessous.

In [ ]:
display(len(data["valeur_fonciere"]))
valeurs = [1.0, 10.0, 1000.0, 5000.0, 10000.0]
for i in valeurs:
    display("Le nombre de valeurs foncières inférieures ou égales à "+str(i)+" est de:")
    display(sum(data["valeur_fonciere"] <= i))

In [ ]:
display(len(data["surface_reelle_bati"]))
valeurs = [1.0, 5.0, 10.0, 30.0]
for i in valeurs:
    display("Le nombre de surfaces bati intérieures ou égales à "+str(i)+" est de:")
    display(sum(data["surface_reelle_bati"] <= i))

On constate que bien qu'il n'y ait pas de valeurs foncières nulle, il y en a **de nombreuses qui sont inférieures ou égales à 10**.
On considère qu'**une transaction est crédible lorsque la valeur foncière est supérieure à 5000** (en-dessous, il s'agit d'une vente qui ne nous intéresse pas).

De même, on considère qu'**une transaction est crédible lorsque la surface du bâtiment est supérieure à 10 m2**.

In [ ]:
# D'abord, on écarte les valeurs foncières inférieures à 5000 et les surface_reelle_bati inférieures à 10 m2

data = data[data["valeur_fonciere"] > 4999]

data = data[data["surface_reelle_bati"] > 9]

In [ ]:
# Il n'y a plus de Dépendances dans le dataset...

# display(data["type_local"].unique())
# data.drop(["code_postal", "longitude", "latitude"], axis = 1).describe()

Par ailleurs, on observe avec le describe ci-dessus que **les écart-types sont très importants par rapport aux moyennes**, surtout pour les variables "valeur_fonciere", "surface_reelle_bati" et "surface_terrain".
Donc, **on enlève les valeurs trop hautes** : ici, cela consiste à *enlever les valeurs dont l'écart à la moyenne en valeur absolue est supérieure à 4 fois l'écart-type*.

In [ ]:
def clean_outliers(data):
    
    # Pour valeur_fonciere :
    data = data[~(np.abs(data['valeur_fonciere'] - data['valeur_fonciere'].mean()) > (4 * data['valeur_fonciere'].std()))]

    # Pour surface_reelle_bati :
    data = data[~(np.abs(data['surface_reelle_bati'] - data['surface_reelle_bati'].mean()) > (4 * data['surface_reelle_bati'].std()))]

    # Pour surface_terrain :
    data = data[~(np.abs(data['surface_terrain'] - data['surface_terrain'].mean()) > (4 * data['surface_terrain'].std()))]
    
    return data

data = clean_outliers(data)

In [ ]:
display(data.drop(["code_postal", "longitude", "latitude"], axis = 1).describe())
boxplot_display(data)

In [ ]:
ax = sns.boxplot(x = "valeur_fonciere", y = "type_local", data = data)

**Ces boxplots correspondent davantage avec la réalité** (médiane des ventes autour de 150 000 euros).